In [31]:
import pandas as pd

In [32]:
INPUT_TSV="/mnt/f/OneDrive/文档（科研）/脚本/Download/9-My-Toolskit/1-下载数据/script/2-ENA/conf/PRJEB49419.tsv"

In [33]:
df = pd.read_csv(INPUT_TSV, sep="\t")
print(df.head())

  run_accession sample_accession                            sample_alias  \
0    ERR9539070    SAMEA13076392  IMED.Italy.Tarquinia_Monterozzi.R10337   
1    ERR9539071    SAMEA13076393  IMED.Italy.Tarquinia_Monterozzi.R10338   
2    ERR9539072    SAMEA13076394  IMED.Italy.Tarquinia_Monterozzi.R10339   
3    ERR9539073    SAMEA13076395  IMED.Italy.Tarquinia_Monterozzi.R10340   
4    ERR9539074    SAMEA13076396  IMED.Italy.Tarquinia_Monterozzi.R10341   

                                      study_title experiment_accession  \
0  Mobility in the Iron Age Central Mediterranean           ERX9079880   
1  Mobility in the Iron Age Central Mediterranean           ERX9079881   
2  Mobility in the Iron Age Central Mediterranean           ERX9079882   
3  Mobility in the Iron Age Central Mediterranean           ERX9079883   
4  Mobility in the Iron Age Central Mediterranean           ERX9079884   

  study_accession  tax_id scientific_name  base_count  \
0      PRJEB49419    9606    Homo sapiens

In [34]:
# 定义一个print_color函数，打印彩色文本
def print_color(text, color_code):
    print(f"\033[{color_code}m{text}\033[0m")

In [35]:
# 检查fastq_ftp列、submitted_ftp列是否为空(只考虑是否全部为空)
# 如果fastq_ftp列不为空但是submitted_ftp列为空，说明只有fastq文件没有bam文件；打印"只有fastq文件没有bam文件"
# 如果fastq_ftp列为空但是submitted_ftp列不为空，说明只有bam文件没有fastq文件；打印"只有bam文件没有fastq文件"
# 如果fastq_ftp列和submitted_ftp列都不为空，说明有fastq文件也有bam文件；打印"有fastq文件也有bam文件"
# 如果fastq_ftp列和submitted_ftp列都为空，说明没有fastq文件也没有bam文件；打印"没有fastq文件也没有bam文件"

# 获取项目名称
project_name = os.path.basename(INPUT_TSV).replace('.tsv', '')
output_dir = os.path.dirname(INPUT_TSV)

if df['fastq_ftp'].isnull().all() and df['submitted_ftp'].isnull().all():
    print_color("[fastq:NO] [bam:NO]", "31")  # 红色
    print_color("请仔细检查该项目是否有数据!", "31")  # 红色
    print_color("建议使用以下脚本重新准备信息:", "31")  # 红色
    print_color("1-下载数据/script/2-ENA/script/0-0-project→全部信息.sh", "31")  # 红色
    print("代码终止执行")
    sys.exit()
    
elif df['fastq_ftp'].isnull().all() and not df['submitted_ftp'].isnull().all():
    print_color("[fastq:NO] [bam:YES]", "32")  # 绿色
    print_color("开始处理BAM文件链接和MD5...", "32")  # 绿色
    
    # 处理submitted_ftp和submitted_md5列
    bam_urls = []
    bai_urls = []
    bam_md5s = []
    bai_md5s = []
    
    for idx, row in df.iterrows():
        if pd.notna(row['submitted_ftp']) and pd.notna(row['submitted_md5']):
            ftp_links = row['submitted_ftp'].split(';')
            md5_values = row['submitted_md5'].split(';')
            
            for ftp, md5 in zip(ftp_links, md5_values):
                if ftp.endswith('.bam'):
                    bam_urls.append(ftp.strip())
                    bam_md5s.append(f"{md5.strip()}  {os.path.basename(ftp.strip())}")
                elif ftp.endswith('.bai'):
                    bai_urls.append(ftp.strip())
                    bai_md5s.append(f"{md5.strip()}  {os.path.basename(ftp.strip())}")
    
    # 写入文件
    with open(f"{output_dir}/{project_name}_bam.url", 'w') as f:
        f.write('\n'.join(bam_urls))
    
    with open(f"{output_dir}/{project_name}_bam_bai.url", 'w') as f:
        f.write('\n'.join(bai_urls))
    
    with open(f"{output_dir}/{project_name}_bam.md5", 'w') as f:
        f.write('\n'.join(bam_md5s))
    
    with open(f"{output_dir}/{project_name}_bam_bai.md5", 'w') as f:
        f.write('\n'.join(bai_md5s))
    
    print_color(f"已生成文件:", "32")
    print_color(f"- {project_name}_bam.url ({len(bam_urls)} 个BAM文件)", "32")
    print_color(f"- {project_name}_bam_bai.url ({len(bai_urls)} 个BAI文件)", "32")
    print_color(f"- {project_name}_bam.md5", "32")
    print_color(f"- {project_name}_bam_bai.md5", "32")
    
elif not df['fastq_ftp'].isnull().all() and df['submitted_ftp'].isnull().all():
    print_color("[fastq:YES] [bam:NO]", "33")  # 黄色
    print_color("警告：只存在FASTQ文件，不建议使用ENA下载器，因为太慢！", "33")
    print_color("建议使用NCBI的SRA工具下载，因为稳定且快！", "33")
    print_color("推荐使用以下文件和脚本:", "33")
    print_color(f"- 配置文件: 1-下载数据/script/2-ENA/conf/{project_name}.txt", "33")
    print_color("- 下载脚本: 1-下载数据/script/1-NCBI/script/3-NCBI-SRA-download.sh", "33")
    
else:
    print_color("[fastq:YES] [bam:YES]", "34")  # 青色
    print_color("检测到FASTQ和BAM文件都存在，建议下载BAM文件！", "34")
    print_color("开始处理BAM文件链接和MD5...", "34")
    
    # 处理submitted_ftp和submitted_md5列
    bam_urls = []
    bai_urls = []
    bam_md5s = []
    bai_md5s = []
    
    for idx, row in df.iterrows():
        if pd.notna(row['submitted_ftp']) and pd.notna(row['submitted_md5']):
            ftp_links = row['submitted_ftp'].split(';')
            md5_values = row['submitted_md5'].split(';')
            
            for ftp, md5 in zip(ftp_links, md5_values):
                if ftp.endswith('.bam'):
                    bam_urls.append(ftp.strip())
                    bam_md5s.append(f"{md5.strip()}  {os.path.basename(ftp.strip())}")
                elif ftp.endswith('.bai'):
                    bai_urls.append(ftp.strip())
                    bai_md5s.append(f"{md5.strip()}  {os.path.basename(ftp.strip())}")
    
    # 写入文件
    with open(f"{output_dir}/{project_name}_bam.url", 'w') as f:
        f.write('\n'.join(bam_urls))
    
    with open(f"{output_dir}/{project_name}_bam_bai.url", 'w') as f:
        f.write('\n'.join(bai_urls))
    
    with open(f"{output_dir}/{project_name}_bam.md5", 'w') as f:
        f.write('\n'.join(bam_md5s))
    
    with open(f"{output_dir}/{project_name}_bam_bai.md5", 'w') as f:
        f.write('\n'.join(bai_md5s))
    
    print_color(f"已生成文件:", "34")
    print_color(f"- {project_name}_bam.url ({len(bam_urls)} 个BAM文件)", "34")
    print_color(f"- {project_name}_bam_bai.url ({len(bai_urls)} 个BAI文件)", "34")
    print_color(f"- {project_name}_bam.md5", "34")
    print_color(f"- {project_name}_bam_bai.md5", "34")

NameError: name 'os' is not defined

In [ ]:
import os
import sys